# Deep Learning Mini-Challenge 1: Image Classification

**Author**: Lukas Reber

**Task**: Train a neural network to classify images from the CIFAR-10 dataset. Focus will be on the training and evaluation of the model, not on the data preprocessing. We will use PyTorch to implement a Multi Layer Perceptron (MLP) and a Convolutional Neural Network (CNN) to classify images the images.

**Dataset**: The [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. There are 50,000 training images and 10,000 test images. The dataset is divided into five training batches and one test batch, each with 10,000 images. The test batch contains exactly 1,000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5,000 images from each class.

## Import dependencies

In [ ]:
import torch